In [ ]:
import os
import sys

src_path = os.path.abspath(os.path.join("../src"))
if src_path not in sys.path:
    sys.path.append(src_path)

import data.make_dataset as make_dataset

make_dataset.main()


In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime

if os.path.isfile("../data/processed/merged.csv"):
    os.remove("../data/processed/merged.csv")

df_bureau = pd.read_csv("../data/processed/bureau.csv")
df_bureau_balance = pd.read_csv("../data/processed/bureau_balance.csv")
df_pos_cash_balance = pd.read_csv("../data/processed/POS_CASH_balance.csv")
df_credit_card_balance = pd.read_csv(
    "../data/processed/credit_card_balance.csv"
)
df_previous_application = pd.read_csv(
    "../data/processed/previous_application.csv"
)
df_installments_payments = pd.read_csv(
    "../data/processed/installments_payments.csv"
)

with pd.read_csv(
    "../data/processed/application_test copy.csv", chunksize=10 ** 3
) as reader:
    for chunk in reader:
        df_merged = pd.merge(chunk, df_bureau, how="left", on="SK_ID_CURR")
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "df_bureau", df_merged.shape)

        df_merged = pd.merge(
            df_merged, df_bureau_balance, how="left", on="SK_ID_BUREAU"
        )
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "df_bureau_balance", df_merged.shape)

        df_merged = pd.merge(
            df_merged,
            df_pos_cash_balance,
            how="left",
            on=["SK_ID_CURR", "MONTHS_BALANCE"],
        )
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "df_pos_cash_balance", df_merged.shape)

        df_merged = pd.merge(
            df_merged,
            df_credit_card_balance,
            how="left",
            on=["SK_ID_PREV", "SK_ID_CURR", "MONTHS_BALANCE"],
        )
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "df_credit_card_balance", df_merged.shape)

        df_merged = pd.merge(
            df_merged,
            df_previous_application,
            how="left",
            on=["SK_ID_PREV", "SK_ID_CURR"],
        )
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "df_previous_application", df_merged.shape)

        df_merged = pd.merge(
            df_merged,
            df_installments_payments,
            how="left",
            on=["SK_ID_PREV", "SK_ID_CURR"],
        )
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "df_installments_payments", df_merged.shape)

        df_merged.to_csv(
            "../data/processed/merged.csv",
            mode="a",
            header=not os.path.isfile("../data/processed/merged.csv"),
            index=False,
        )
        df_merged.dropna(
            thresh=np.floor(0.1 * len(df_merged.columns)), inplace=True
        )
        df_merged.drop_duplicates(inplace=True)
        print(datetime.now(), "write", df_merged.shape)
        del df_merged
